In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import os
import sys
from multiprocessing import Pool, cpu_count
from tqdm.auto import tqdm
import pandas as pd

In [3]:
import warnings
warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)

In [4]:
project_root = os.path.abspath(
    os.path.join("C:/Users/jllgo/OneDrive/Documentos/UFMG/MSc & MSI 2/MatchSegmentation")
)
if project_root not in sys.path:
    sys.path.append(project_root)

In [5]:
RAW_DATA_PATH = '../data/raw/PL-22-23'
INTERMEDIATE_DATA_PATH = '../data/intermediate/PL-22-23'
PROCESSED_DATA_PATH = "../data/processed/PL-22-23"
CSV_DATA_PATH = "../data/csv"

In [6]:
pre_processed_ids = os.listdir(INTERMEDIATE_DATA_PATH)
pre_processed_ids = [int(x.split(".")[0]) for x in pre_processed_ids]
len(pre_processed_ids)

175

In [7]:
ok = [4448,
 4493,
 4502,
 4551,
 4556,
 4566,
 4567,
 4570,
 4572,
 4574,
 4578,
 4579,
 4618,
 4619]

In [8]:
from src.pass_networks.process_games import load_game

In [10]:
matches_events = []
if len(pre_processed_ids) > 0:
    for game_id in tqdm(pre_processed_ids, desc="Processing Games", total=len(pre_processed_ids)):
        metadata_df, players_df, events_df = load_game((INTERMEDIATE_DATA_PATH, game_id))
        matches_events.append(events_df)

Processing Games:   0%|          | 0/175 [00:00<?, ?it/s]

In [11]:
matches_events = pd.concat(matches_events, axis=0).reset_index(drop=True)

In [12]:
matches_events = matches_events[matches_events['possession_id'].notna()].reset_index(drop=True)
grouped = matches_events.groupby(['match_id', 'possession_type']).size().unstack().reset_index().reset_index(drop=True)
grouped

possession_type,match_id,CARRY,PASS,SHOT
0,4436,32,1020,19
1,4438,39,900,22
2,4439,30,949,27
3,4440,31,1060,24
4,4441,46,804,30
...,...,...,...,...
170,4621,28,994,30
171,4622,30,818,35
172,4623,22,958,26
173,4624,22,1006,27


In [13]:
totals = grouped.drop(columns=['match_id']).sum()
means = grouped.drop(columns=['match_id']).mean()

# Exibindo os resultados
print("Totais por coluna:")
print(totals)

print("\nMédias por coluna:")
print(means)

Totais por coluna:
possession_type
CARRY      4747
PASS     164278
SHOT       4491
dtype: int64

Médias por coluna:
possession_type
CARRY     27.125714
PASS     938.731429
SHOT      25.662857
dtype: float64


In [14]:
from src.pass_networks.process_match_info import get_match_info
players_info, teams_info, games_info = get_match_info(CSV_DATA_PATH)

In [24]:
len(games_info[(games_info['home_team_id']==11) | (games_info['away_team_id']==11)])

19

In [32]:
4491 / (164278+4747+4491)

0.02588233938080638